# Import

In [1]:
// %install-swiftpm-flags -c release
// %install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.5")' SwiftCoreMLTools
// %install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

In [2]:
import Foundation
import TensorFlow
// import SwiftCoreMLTools
// import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Categorize Categorical Features with Ordinal values

In [6]:
var categoricalValues = Array(repeating: Set<Int32>(), count: 2)

for record in categoricalFeatures {
    categoricalValues[0].insert(record[0])
    categoricalValues[1].insert(record[1])
}

let sortedCategoricalValues = [categoricalValues[0].sorted(), categoricalValues[1].sorted()]

let ordinalCategoricalFeatures = categoricalFeatures.map{ [Int32(sortedCategoricalValues[0].firstIndex(of:$0[0])!), 
                                                           Int32(sortedCategoricalValues[1].firstIndex(of:$0[1])!)] }

## Split Train and Test

In [7]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(ordinalCategoricalFeatures[0..<numTrainRecords]))
let xCategoricalAllTest = matrixTranspose(Array(ordinalCategoricalFeatures[numTrainRecords...]))
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, 1]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, 1]))
}

let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [8]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [9]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [10]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 1] [405, 1] [405, 1]
Testing shapes  [101, 11] [101, 1] [101, 1] [101, 1]


# Model

In [11]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
    var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var embeddingLayer = Dense<Float>(inputSize: (2 + 5), outputSize: 32, activation: relu)
    var allInputConcatLayer = Dense<Float>(inputSize: (32 + 32), outputSize: 128, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let numericalInput = numericalLayer(input.numerical[0])
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
        let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        let embeddingInput = embeddingLayer(embeddingConcat)
        let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [12]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [13]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [14]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 602.2465, MAE: 22.243177
Epoch 2: MSE: 457.58795, MAE: 18.718084
Epoch 3: MSE: 302.8213, MAE: 14.521089
Epoch 4: MSE: 179.62434, MAE: 10.642439
Epoch 5: MSE: 117.44141, MAE: 7.8646207
Epoch 6: MSE: 91.96334, MAE: 6.557596
Epoch 7: MSE: 77.76145, MAE: 5.928409
Epoch 8: MSE: 69.38946, MAE: 5.575482
Epoch 9: MSE: 60.062626, MAE: 5.1242743
Epoch 10: MSE: 55.129135, MAE: 4.9145975
Epoch 11: MSE: 49.095047, MAE: 4.5226054
Epoch 12: MSE: 46.168064, MAE: 4.2295365
Epoch 13: MSE: 40.14866, MAE: 3.9058523
Epoch 14: MSE: 38.224552, MAE: 3.7630463
Epoch 15: MSE: 37.28929, MAE: 3.7424583
Epoch 16: MSE: 33.70654, MAE: 3.483003
Epoch 17: MSE: 30.183, MAE: 3.3566082
Epoch 18: MSE: 30.318037, MAE: 3.19677
Epoch 19: MSE: 27.617678, MAE: 3.1030161
Epoch 20: MSE: 27.005451, MAE: 3.1135278
Epoch 21: MSE: 26.5478, MAE: 3.0210347
Epoch 22: MSE: 24.17986, MAE: 2.9339128
Epoch 23: MSE: 24.320946, MAE: 2.8776896
Epoch 24: MSE: 24.360575, MAE: 2.835313
Epoch 25: MSE: 23.631884, MAE: 2.817111
Epoch 

Epoch 205: MSE: 5.880944, MAE: 1.5475534
Epoch 206: MSE: 5.8070827, MAE: 1.518448
Epoch 207: MSE: 4.6575966, MAE: 1.4772357
Epoch 208: MSE: 4.8595395, MAE: 1.5747473
Epoch 209: MSE: 5.7451315, MAE: 1.5626036
Epoch 210: MSE: 4.802931, MAE: 1.488935
Epoch 211: MSE: 4.7372546, MAE: 1.6115551
Epoch 212: MSE: 4.8735623, MAE: 1.5552605
Epoch 213: MSE: 4.965569, MAE: 1.5626185
Epoch 214: MSE: 4.949934, MAE: 1.5914142
Epoch 215: MSE: 4.761629, MAE: 1.4923145
Epoch 216: MSE: 5.2863564, MAE: 1.5494277
Epoch 217: MSE: 5.452098, MAE: 1.5343617
Epoch 218: MSE: 4.704102, MAE: 1.4689873
Epoch 219: MSE: 4.7844157, MAE: 1.5427825
Epoch 220: MSE: 4.772698, MAE: 1.4759393
Epoch 221: MSE: 4.791302, MAE: 1.5122102
Epoch 222: MSE: 4.6984563, MAE: 1.4432056
Epoch 223: MSE: 4.504805, MAE: 1.4963568
Epoch 224: MSE: 5.2656503, MAE: 1.5206345
Epoch 225: MSE: 4.5683894, MAE: 1.4971664
Epoch 226: MSE: 4.993668, MAE: 1.5055715
Epoch 227: MSE: 5.3986897, MAE: 1.5446045
Epoch 228: MSE: 4.12638, MAE: 1.4279929
Epoch 2

Epoch 404: MSE: 2.1640046, MAE: 1.2087833
Epoch 405: MSE: 3.1159043, MAE: 1.3962429
Epoch 406: MSE: 2.6449168, MAE: 1.173127
Epoch 407: MSE: 1.9852587, MAE: 1.001127
Epoch 408: MSE: 2.3976362, MAE: 1.4179908
Epoch 409: MSE: 2.763637, MAE: 1.2825129
Epoch 410: MSE: 3.4900057, MAE: 1.2296029
Epoch 411: MSE: 2.236205, MAE: 1.0969621
Epoch 412: MSE: 2.8581505, MAE: 1.2296846
Epoch 413: MSE: 2.51894, MAE: 1.2596207
Epoch 414: MSE: 2.100371, MAE: 1.1416446
Epoch 415: MSE: 2.9191525, MAE: 1.3161455
Epoch 416: MSE: 2.4577818, MAE: 1.0809426
Epoch 417: MSE: 2.454856, MAE: 1.3252437
Epoch 418: MSE: 2.1913981, MAE: 1.2109532
Epoch 419: MSE: 2.261903, MAE: 1.245345
Epoch 420: MSE: 2.57616, MAE: 1.3794576
Epoch 421: MSE: 2.6203027, MAE: 1.3636689
Epoch 422: MSE: 2.5898848, MAE: 1.3182193
Epoch 423: MSE: 2.86727, MAE: 1.1727836
Epoch 424: MSE: 2.5254514, MAE: 1.1456196
Epoch 425: MSE: 2.2893362, MAE: 1.186591
Epoch 426: MSE: 1.916116, MAE: 1.279881
Epoch 427: MSE: 3.468405, MAE: 1.3659574
Epoch 428:

# Test

In [15]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.46891883, MAE: 0.05201415
